In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import math
from sklearn.neighbors import BallTree as BallTree
import matplotlib.pyplot as plt

In [ ]:
bath = xr.open_dataset(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\ERA5_metOcean_bathy.nc')
depthNS = 20
shp_coast2 = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\Continental shapefile coordinates2.xlsx')
ERA5_on_coast = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\ERA5_locs_on_coast_fin.xlsx')
ERA5_around_coast = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\ERA5_coordinates_around_coast_fin.xlsx')
X1 = ERA5_on_coast.iloc[:,0]; Y1 = ERA5_on_coast.iloc[:,1]
X2 = ERA5_around_coast.iloc[:,0]; Y2 = ERA5_around_coast.iloc[:,1]
lfhs20 = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\lf_waveheight_lookuptable_d20.xlsx',header=None)
lftm20 = pd.read_excel(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\lf_waveperiod_lookuptable_d20.xlsx',header=None)
pi = math.pi
grav = 9.81
seasons = np.array(['DJF', 'MAM', 'JJA', 'SON'])

In [ ]:
cols = ['lon', 'lat', 'depth0','shoreN','tm','tp','Ks','Kr','wind50','wind95','windAv','swh50','swh95','swhAv','ig50','ig95','igAv','wavePo','windAvUp','swhAvUp','igAvUp','operability','frequency','duration']
df = pd.DataFrame(columns = cols, index=range(0,4560))

In [ ]:
# THRESHOLDS
Twind = 13.8
Twave = 2.0
Tig = 0.05

In [ ]:
for i in range(0,4560):
    print(i)
    # LOAD ERA5 DATA
    ds = xr.open_dataset(r'c:\Users\wiegel\OneDrive - Stichting Deltares\Documents\input\ERA5_output_world_coast\Loc%s.nc' % i)
    dfx = ds.to_dataframe()
    dfx = dfx.dropna()
    if dfx.empty: continue
    
    dfx['season'] = seasons[(dfx.index.month // 3) % 4]
    df.loc[i].lon = dfx.longitude[0]
    df.loc[i].lat = dfx.latitude[0]
    
    # DEPTH OFFSHORE
    depth = bath.sel(latitude=dfx.latitude[0], longitude=dfx.longitude[0], method='nearest')
    depth = depth.to_dataframe()
    df['depth0'].loc[i] = depth.wmb[0]
    if np.isnan(df.loc[i].depth0) or df.loc[i].depth0 <= depthNS: df.loc[i].depth0 = depthNS
     
    # SHORENORMAL ORIENTATION
    F = np.array([df.loc[i].lon,df.loc[i].lat]).reshape(1,-1)
    x1 = shp_coast2.iloc[:,0].tolist()
    y1 = shp_coast2.iloc[:,1].tolist()
    X = np.column_stack((x1,y1))
    tree = BallTree(X, leaf_size=100)             
    dist, ind = tree.query(F, k=1)
    Cordmin = (X[ind-4]+X[ind-3]+X[ind-2]+X[ind-1])/4
    Cordpls = (X[ind+4]+X[ind+3]+X[ind+2]+X[ind+1])/4
    coastcoord = (Cordmin+Cordpls)/2
    theta = math.atan2((df.loc[i].lon - coastcoord[0,0,0]),(df.loc[i].lat - coastcoord[0,0,1]))
    shoreN = (theta*180/(math.pi)) % 360
    if np.isnan(shoreN):
        theta = math.atan2(X2[i]-X1[i],Y2[i]-Y1[i])
        shoreN = (theta*180/(math.pi)) % 360
    df['shoreN'].loc[i] = shoreN
    
    # WIND
    wind = np.sqrt(dfx.u10**2 + dfx.v10**2); wind[-1] = 0; wind[0] = 0
    dfx['wind'] = wind
    df['wind50'].loc[i] = wind.quantile([0.50]).iloc[0]
    df['wind95'].loc[i] = wind.quantile([0.95]).iloc[0]
    df['windAv'].loc[i] = np.count_nonzero(wind < Twind)/len(wind)*100
    idx = np.argwhere(np.diff(np.sign(Twind - wind))).flatten()
    Events = idx.reshape(len(idx)//2,2)
    if idx.size != 0:
        IntervalIndex = np.column_stack((Events[:-1,1],Events[1:,0]))
        ShortIntervalIndex = IntervalIndex[Events[1:,0]-Events[:-1,1] <= 4]
        for n in range(0,len(ShortIntervalIndex)):
            x = np.arange(ShortIntervalIndex[n,0]+1,ShortIntervalIndex[n,1]+1)
            wind[x] = Twind
        dfx.wind = wind
        df.loc[i].windAvUp = np.count_nonzero(wind < Twind)/len(wind)*100
    else:
        df.loc[i].windAvUp = 100
        
    # NEARSHORE TRANSLATION
    dirWave = dfx.mwd
    waveHeight = dfx.swh    
    wavePeriod = dfx.mwp
    peakPeriod = dfx.pp1d
    dirWave = dirWave * 1.
    waveHeight = waveHeight.astype('float')
    wavePeriod = wavePeriod * 1.
    depthOffshore = float(df.loc[i].depth0)
    depthNearshore = float(depthNS)
    shoreNormal = float(shoreN % 360)
    df.loc[i].tp = np.mean(peakPeriod)
    
    # WAVE LENGHT/NUMBER (DEEP & NEAR) BASED ON THE WAVE PERIOD WHICH REMAINS CONSTANT
    # Calculation based on the Matlab function disper.m and transformed to nearshore
    
    w2 = (2 * pi / wavePeriod)**2 * depthOffshore / grav
    q = w2 / ( (1 - np.exp(-(w2**(5/4))))**(2/5))
    for j in range(2):
        thq = np.tanh(q)
        thq2 = 1 - thq**2
        a = (1 - q * thq)*thq2
        b = thq + q * thq2
        c = q * thq - w2
        arg = b**2 - (4*a*c)
        arg = (-b + np.sqrt(arg)) / (2 * a)
        iq = np.abs(a * c) < (10**-8 * b**2)
        arg[iq] = - c[iq] / b[iq]
        q = q + arg
    k = q / depthOffshore
    Ldeep = 2 * pi / k
    
    # Nearshore depth wave number and wave length calculation via dispersion relation
    w3 = (2 * pi / wavePeriod)**2 * depthNearshore / grav
    q1 = w3 / (1 - np.exp(-(w3**(5/4))))**(2/5)
    for m in range(2):
        thq = np.tanh(q1)
        thq2 = 1 - thq**2
        a = (1 - q1 * thq)*thq2
        b = thq + q1 *thq2
        c = q1 * thq - w3
        arg = b**2 - (4*a*c)
        arg = (-b + np.sqrt(arg)) / (2 * a)
        iq = np.abs(a * c) < (10**-8 * b**2)
        arg[iq] = - c[iq] / b[iq]
        q1 = q1 + arg
    k1 = q1 / depthNearshore
    Lnear = 2 * pi / k1
  
    # Calculate the waveVelocity (deep and near)
    Cdeep = ((grav/k) * np.tanh(k * depthOffshore))**0.5
    Cnear = ((grav/k1) * np.tanh(k1 * depthNearshore))**0.5
    
    # Calculate the wave-angle of the point offshore
    relDir = (dirWave - shoreNormal + 180.) % 360. - 180.
    SinThetadeep = np.sin(relDir * (pi/180.))
    
    # Calculate the wave-angle of the point near the coast
    SinThetanear = (SinThetadeep / Cdeep) * Cnear
    relDirnear = np.arcsin(SinThetanear) * (180./pi)
    
    # Refraction coefficient
    Kr = (np.cos(relDir * (pi/180.)) / np.cos(relDirnear * (pi/180.)))**0.5
    Kr[(Kr < 0.8)] = 0.8
    df.loc[i].Kr = np.mean(Kr)
    
    # Calculate group wave velocities
    waveNumberdeep = 2*pi / Ldeep
    if depthOffshore/Ldeep.any() > 0.5: ndeep = 0.5
    else: ndeep = 0.5 * (1 + (2 * waveNumberdeep * depthOffshore) / np.sinh(2 * waveNumberdeep * depthOffshore))
    waveVelocitydeep = ( (grav*Ldeep / (2*pi)) * np.tanh(2*pi * depthOffshore / Ldeep) )**0.5
    Cgroupdeep = ndeep * waveVelocitydeep
    waveNumbernear = 2*pi / Lnear
    nnear = 0.5 * (1 + (2 * waveNumbernear * depthNearshore) / np.sinh(2 * waveNumbernear * depthNearshore))
    waveVelocitynear = ( (grav*Lnear / (2*pi)) * np.tanh(2*pi * depthNearshore / Lnear) )**0.5
    Cgroupnear = nnear * waveVelocitynear
    
    # Shoaling coefficient:
    Ks = (Cgroupdeep / Cgroupnear) ** 0.5
    df.loc[i].Ks = np.mean(Ks)
    
    # Wave-climate of the near shore
    waveHeightnearAlldir = Ks * waveHeight
    waveHeightnearAlldir[waveHeightnearAlldir > (depthNearshore*0.73)] = depthNearshore*0.73
    dfx['swh_nr_ad'] = waveHeightnearAlldir
    waveHeightnearReldir = Kr * Ks * waveHeight
    waveHeightnearReldir[waveHeightnearReldir > (depthNearshore*0.73)] = depthNearshore*0.73
    dfx['swh_nr_rd'] = waveHeightnearReldir
    wavePeriodnear = wavePeriod * 1.
    df.loc[i].tm = np.mean(wavePeriodnear)
    ixD = ~((-90 < relDir) & (relDir < 90))
    waveHeightnearReldir[ixD] = float('NaN')
    waveHeightnearReldir[ixD] = float('NaN')
    relDirnear[ixD] = relDir[ixD]
    df['swh50'].loc[i] = waveHeightnearReldir.quantile([0.5]).iloc[0]
    df['swh95'].loc[i] = waveHeightnearReldir.quantile([0.95]).iloc[0]
    waveHeightnearReldir = waveHeightnearReldir.fillna(0)
    df['swhAv'].loc[i] = np.count_nonzero(waveHeightnearReldir < Twave)/len(waveHeightnearReldir)*100
    waveHeightnearReldir[-1] = 0; waveHeightnearReldir[0] = 0
    idx = np.argwhere(np.diff(np.sign(Twave - waveHeightnearReldir))).flatten()
    Events = idx.reshape(len(idx)//2,2)
    if idx.size != 0:
        IntervalIndex = np.column_stack((Events[:-1,1],Events[1:,0]))
        ShortIntervalIndex = IntervalIndex[Events[1:,0]-Events[:-1,1] <= 4]
        for n in range(0,len(ShortIntervalIndex)):
            x = np.arange(ShortIntervalIndex[n,0]+1,ShortIntervalIndex[n,1]+1)
            waveHeightnearReldir[x] = Twave
        dfx.swh_nr_rd = waveHeightnearReldir
        df.loc[i].swhAvUp = np.count_nonzero(waveHeightnearReldir < Twave)/len(waveHeightnearReldir)*100
    else:
        df.loc[i].swhAvUp = 100
   
    # Infragravity (bound) waves nearshore
    idhs = (dfx.swh_nr_ad * 20).round().astype(int)
    idhs[idhs > 249]=249
    idtp = (peakPeriod * 10).round().astype(int)
    idtp[idtp > 249]=249
    lfhs = np.array(lfhs20)
    lftm = np.array(lftm20)
    lfhs = lfhs[idhs,idtp]
    lftm = lftm[idhs,idtp]
    lfhs[-1] = 0; lfhs[0] = 0
    dfx['lfhs'] = lfhs
    dfx['lftm'] = lftm
    df['ig50'].loc[i] = dfx.lfhs.quantile([0.50]).iloc[0]
    df['ig95'].loc[i] = dfx.lfhs.quantile([0.95]).iloc[0]
    df['igAv'].loc[i] = np.count_nonzero(lfhs < Tig)/len(lfhs)*100
    idx = np.argwhere(np.diff(np.sign(Tig - lfhs))).flatten()
    Events = idx.reshape(len(idx)//2,2)
    if idx.size != 0:
        IntervalIndex = np.column_stack((Events[:-1,1],Events[1:,0]))
        ShortIntervalIndex = IntervalIndex[Events[1:,0]-Events[:-1,1] <= 4]
        for n in range(0,len(ShortIntervalIndex)):
            lfhs[np.arange(ShortIntervalIndex[n,0]+1,ShortIntervalIndex[n,1]+1)] = Tig
        dfx.lfhs = lfhs
        df.loc[i].igAvUp = np.count_nonzero(lfhs < Tig)/len(lfhs)*100
    else:
        df.loc[i].igAvUp = 100
    
    # Longshore Sediment Transport potential
    dfx['wavePo'] = waveHeightnearAlldir**2.5*np.sin(2*SinThetanear)
    df['wavePo'].loc[i] = np.sum(np.abs(dfx.wavePo))/len(waveHeightnearAlldir)
    
    #Calculate the operability
    dfy = pd.concat([dfx.wind >= Twind, dfx.swh_nr_rd >= Twave, dfx.lfhs >= Tig], axis=1)
    persistence = dfy.sum(axis=1); persistence[-1]=0; persistence[0]=0
    idx3 = np.argwhere(np.diff(persistence > 0)).flatten()
    Events3 = idx3.reshape(len(idx3)//2,2)
    if idx3.size != 0:
        IntervalIndex = np.column_stack((Events3[:-1,1],Events3[1:,0]))
        ShortIntervalIndex = IntervalIndex[Events3[1:,0]-Events3[:-1,1] <= 4]
        for n in range(0,len(ShortIntervalIndex)):
            persistence[np.arange(ShortIntervalIndex[n,0]+1,ShortIntervalIndex[n,1]+1)] = 4
        dfx['persistence'] = persistence
    #dfz = dfy[dfy.wind | dfy.swh_nr_rd | dfy.lfhs]
    df['operability'].loc[i] = (len(dfy.index) - len(dfx.persistence.loc[~(dfx.persistence==0)]))/len(dfy.index)*100
    
    #Calculate the frequency (per year) shut down and restart again limits the uptime
    freq = dfy.sum(axis=1); freq[-1]=0; freq[0]=0
    if np.sum(freq) != 0:
        idx2 = np.argwhere(np.diff(freq > 0)).flatten()
        df['frequency'].loc[i] = len(idx2.reshape(len(idx2)//2,2))/40
    
    #Calculate the duration (per event) short periods could be caught up more quickly
        Events2 = idx2.reshape(len(idx2)//2,2)
        df['duration'].loc[i] = np.sum(Events2[:,1]-Events2[:,0])/len(Events2)
    else: df['frequency'].loc[i] = 0; df['duration'].loc[i] = 0

In [ ]:
df.to_excel('output_run20_4560locs_d20_w138_s_2_i_005.xlsx')